# E-commerce data

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv("./data/Ecommerce_Customers.csv", inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [13]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler 

In [15]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [19]:
assembler = VectorAssembler(
    inputCols = [
        'Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership'
    ],
    outputCol='features'
)

In [20]:
output = assembler.transform(data)  # No train test split yet

In [21]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [22]:
final_data = output.select('features', 'Yearly Amount Spent')

In [23]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                363|
|   mean| 499.51260794250715|
| stddev|  81.41149374801284|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [26]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                137|
|   mean| 498.78790106690104|
| stddev|  73.75432581273087|
|    min|  304.1355915788555|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [29]:
lr_model = lr.fit(train_data)

In [30]:
test_results = lr_model.evaluate(test_data)

In [31]:
test_results.residuals.show()   # The difference between predicted and actual

+-------------------+
|          residuals|
+-------------------+
|0.45262769067102226|
| -4.988134666353233|
| 6.7330355322067135|
|    5.0716634982399|
| -6.459817998160645|
| 1.0743107825533116|
| -4.435001595290942|
|  -6.70426011075017|
| -5.136066523673435|
|-14.698881589057464|
|  -5.47226083397419|
|-11.172832455619982|
|  7.891272504816868|
| -9.621965364361074|
| 0.9948470479002935|
| -2.554174063368464|
|  -8.91193908321037|
|-13.887650226775577|
|  4.797676155219392|
|  4.847969779473374|
+-------------------+
only showing top 20 rows



In [32]:
test_results.rootMeanSquaredError  # root of sum of square of residuals 

9.435147064222024

In [33]:
test_results.r2     # we explain 98% of the variance in the data

0.9835144283404165

In [35]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [36]:
unlabeled_data = test_data.select('features')

In [37]:
predictions = lr_model.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.61178606739463|
|[30.8364326747734...|472.49003509334284|
|[30.9716756438877...|  487.905574224686|
|[31.0472221394875...| 387.4257356907815|
|[31.0613251567161...|494.01527605606225|
|[31.3895854806643...| 408.9953002774296|
|[31.4252268808548...| 535.2017202500529|
|[31.4474464941278...| 425.3070022059742|
|[31.5147378578019...|494.94855452013485|
|[31.5741380228732...| 559.1081537496443|
|[31.6253601348306...| 381.8091615908984|
|[31.8093003166791...| 547.9447318184612|
|[31.8209982016720...| 416.7840085083965|
|[31.8279790554652...| 449.6247129113026|
|[31.8293464559211...| 384.1574909400747|
|[31.8530748017465...|461.83929752572044|
|[31.8854062999117...| 399.0152120556859|
|[31.9365486184489...|441.08703512210377|
|[31.9480174211613...| 457.1232007376784|
|[32.0215955013870...|  516.724204978354|
+--------------------+------------

In [40]:
predictions.describe().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|               137|
|   mean| 499.4618520315426|
| stddev| 73.48239547410215|
|    min|313.98405652874635|
|    max| 767.0341737273609|
+-------+------------------+

